# Fetch the Mnist_784 dataset

In [1]:
from sklearn.datasets import fetch_openml
import numpy as np
import json
import requests
X, y = fetch_openml('mnist_784', return_X_y=True, parser='auto')

In [2]:
X.shape

(70000, 784)

Deploy the model in your Data Science project using RHODS UI under "Models and model servers"

# Submit HTTP REST request to the ModelMesh for single sample

In [3]:
model_name="mnist123"

import requests
import json
URL='http://modelmesh-serving.huggingface.svc.cluster.local:8008/v2/models/'+model_name+'/infer' # underscore characters are removed
headers = {}
payload = {
        "inputs": [{ "name": "input_0", "shape": (1,1,28,28), "datatype": "FP32", "data": X.iloc[0].values.tolist()}]
    }
headers = {"content-type": "application/json"}
res = requests.post(URL, json=payload, headers=headers)
print(res)
print(res.text)

<Response [200]>
{"model_name":"mnist123__isvc-8d6ab6dcea","model_version":"1","outputs":[{"name":"output_0","datatype":"FP32","shape":[1,10],"data":[-814.69055,-957.3951,-849.8701,-50.25647,-1386.2316,0,-1192.0592,-917.001,-584.47845,-549.07214]}]}


In [4]:
print("Expected",y.iloc[0],"Actual",np.argmax(res.json()['outputs'][0]['data']))

Expected 5 Actual 5


# Submit HTTP REST request to the ModelMesh for a batch of samples

In [5]:
model_name="mnist123"

import requests
import json
URL='http://modelmesh-serving.huggingface.svc.cluster.local:8008/v2/models/'+model_name+'/infer' # underscore characters are removed
headers = {}
payload = {
        "inputs": [{ "name": "input_0", "shape": (5,1,28,28), "datatype": "FP32", "data": X.loc[0:4].values.flatten().tolist()}]
    }
#print(payload)
headers = {"content-type": "application/json"}
res = requests.post(URL, json=payload, headers=headers)
print(res)
print(res.text)

<Response [200]>
{"model_name":"mnist123__isvc-8d6ab6dcea","model_version":"1","outputs":[{"name":"output_0","datatype":"FP32","shape":[5,10],"data":[-814.69055,-957.39514,-849.8701,-50.25641,-1386.2317,0,-1192.0592,-917.00104,-584.47845,-549.07214,0,-1738.2633,-1001.20197,-1479.0795,-1671.8254,-1337.5544,-1102.626,-1238.0874,-1310.7225,-1436.8561,-1826.0315,-1556.1779,-1231.7928,-1325.793,0,-1360.6669,-1473.7661,-1042.4226,-1417.6643,-456.1117,-1381.7842,0,-658.9868,-1014.41394,-828.6969,-1272.7472,-1026.6971,-779.3543,-694.4404,-1050.7561,-1771.2993,-1376.7881,-1287.1708,-1171.8777,-408.81616,-1147.683,-1725.4868,-665.1205,-822.9125,0]}]}


In [6]:
actual=y[0:5].values.tolist()
expected=np.argmax(np.array(res.json()['outputs'][0]['data']).reshape(res.json()['outputs'][0]['shape']),axis=1)
print(actual,expected)

['5', '0', '4', '1', '9'] [5 0 4 1 9]


# Submit gRPC request to the ModelMesh for single sample

In [7]:
!pip install grpcio grpcio-tools==1.46.0


[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [8]:
#!wget https://raw.githubusercontent.com/kserve/kserve/master/docs/predict-api/v2/grpc_predict_v2.proto
!wget https://raw.githubusercontent.com/kserve/modelmesh-serving/main/fvt/proto/kfs_inference_v2.proto
!python3 -m grpc_tools.protoc -I. --python_out=. --grpc_python_out=. ./kfs_inference_v2.proto

--2023-06-09 15:56:52--  https://raw.githubusercontent.com/kserve/modelmesh-serving/main/fvt/proto/kfs_inference_v2.proto
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10394 (10K) [text/plain]
Saving to: ‘kfs_inference_v2.proto’

kfs_inference_v2.pr 100%[===================>]  10.15K  --.-KB/s    in 0s      

2023-06-09 15:56:52 (87.6 MB/s) - ‘kfs_inference_v2.proto’ saved [10394/10394]



In [9]:
model_name="mnist123"
payload = { "model_name": model_name,
            "inputs": [{ "name": "input_0", "shape": (1,1,28,28), "datatype": "FP32", "contents": {"fp32_contents":X.iloc[0].values.tolist()}}]
    }

In [10]:
import grpc
import kfs_inference_v2_pb2, kfs_inference_v2_pb2_grpc
grpc_url="modelmesh-serving.huggingface.svc.cluster.local:8033"
request=kfs_inference_v2_pb2.ModelInferRequest(model_name=model_name,inputs=payload["inputs"])
grpc_channel = grpc.insecure_channel(grpc_url)
grpc_stub = kfs_inference_v2_pb2_grpc.GRPCInferenceServiceStub(grpc_channel)
response = grpc_stub.ModelInfer(request)

In [11]:
print(type(response.outputs),type(response.raw_output_contents))
from google.protobuf.json_format import MessageToDict
d = MessageToDict(response.outputs[0])
print(d)
binary_data=bytes([x for x in response.raw_output_contents[0]])

<class 'google.protobuf.pyext._message.RepeatedCompositeContainer'> <class 'google.protobuf.pyext._message.RepeatedScalarContainer'>
{'name': 'output_0', 'datatype': 'FP32', 'shape': ['1', '10']}


In [12]:
import struct
import base64
FLOAT = 'f'
fmt = '<' + FLOAT * (len(binary_data) // struct.calcsize(FLOAT))
print("Expected",y.iloc[0],"Actual",np.argmax(np.array(struct.unpack(fmt, binary_data))))

Expected 5 Actual 5


# Submit gRPC request to the ModelMesh for batch of samples

In [13]:
model_name="mnist123"
payload = { "model_name": model_name,
            "inputs": [{ "name": "input_0", "shape": (5,1,28,28), "datatype": "FP32", "contents": {"fp32_contents":X.loc[0:4].values.flatten().tolist()}}]
    }

In [14]:
import grpc
import kfs_inference_v2_pb2, kfs_inference_v2_pb2_grpc
grpc_url="modelmesh-serving.huggingface.svc.cluster.local:8033"
request=kfs_inference_v2_pb2.ModelInferRequest(model_name=model_name,inputs=payload["inputs"])
grpc_channel = grpc.insecure_channel(grpc_url)
grpc_stub = kfs_inference_v2_pb2_grpc.GRPCInferenceServiceStub(grpc_channel)
response = grpc_stub.ModelInfer(request)

In [15]:
print(type(response.outputs),type(response.raw_output_contents))
from google.protobuf.json_format import MessageToDict
d = MessageToDict(response.outputs[0])
print(d)
binary_data=bytes([x for x in response.raw_output_contents[0]])

<class 'google.protobuf.pyext._message.RepeatedCompositeContainer'> <class 'google.protobuf.pyext._message.RepeatedScalarContainer'>
{'name': 'output_0', 'datatype': 'FP32', 'shape': ['5', '10']}


In [16]:
import struct
import base64
FLOAT = 'f'
fmt = '<' + FLOAT * (len(binary_data) // struct.calcsize(FLOAT))
numbers = [str(n) for n in np.argmax(np.array(struct.unpack(fmt, binary_data)).reshape(*[int(shapeval) for shapeval in d['shape']]),axis=1)]
print("Expected",y[0:5].values.tolist(),"Actual",numbers)

Expected ['5', '0', '4', '1', '9'] Actual ['5', '0', '4', '1', '9']
